# Window Functions: Ranking Functions

Reference [Introduction to T-SQL Window Functions - Simple Talk (red-gate.com)](https://www.red-gate.com/simple-talk/sql/t-sql-programming/introduction-to-t-sql-window-functions/)  

### ROW\_NUMBER  

One benefit of ROW\_NUMBER is the ability to turn non-unique rows into unique rows. This could be used to eliminate duplicate rows, for example.

In [ ]:
CREATE TABLE #Duplicates (Col1 INT, Col2 CHAR(1));

INSERT INTO #Duplicates (Col1, Col2)
VALUES (1, 'A'), (2, 'B'), (2, 'B'), (2, 'B'), (3, 'C'), (4, 'D'), (4, 'D'), (5, 'E'), (5, 'E'), (5, 'E');

SELECT *
FROM #Duplicates;

Adding ROW\_NUMBER and partitioning by each column will restart the row numbers for each unique set of rows. You can identify the unique rows by finding those with a row number equal to one.

In [ ]:
DELETE T
FROM
(
    SELECT *
    , DupRank = ROW_NUMBER() OVER (
                PARTITION BY key_value
                ORDER BY (SELECT NULL)
                )
    FROM original_table
) AS T
WHERE DupRank > 1;
GO

Now, all you have to do is to delete any rows that have a row number greater than one. To do this, you must delete the rows from a CTE as you cannot add window functions to a WHERE clause.

In [ ]:
WITH Dupes
AS (
    SELECT Col1
        , Col2
        , ROW_NUMBER() OVER (
            PARTITION BY Col1
            , Col2 ORDER BY Col1
            ) AS RowNum
    FROM #Duplicates
    )
DELETE Dupes
WHERE RowNum <> 1;

SELECT *
FROM #Duplicates;


A WHERE clause can also be used to view _n_ nmber of duplicate records from within a set.

In [ ]:
WITH Dupes
AS (
    SELECT Col1
        , Col2
        , ROW_NUMBER() OVER (
            PARTITION BY Col1
            , Col2 ORDER BY Col1
            ) AS RowNum
    FROM #Duplicates
    )
SELECT *
FROM Dupes
WHERE RowNum <= 3;
